## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,0,Hermanus,ZA,-34.4187,19.2345,51.91,88,100,3.65,light rain
1,1,Cape Town,ZA,-33.9258,18.4232,53.89,89,75,1.01,light rain
2,2,Ushuaia,AR,-54.8000,-68.3000,40.66,70,20,1.14,few clouds
3,3,Brae,GB,60.3964,-1.3530,53.51,93,97,11.54,overcast clouds
4,4,Tessalit,ML,20.1986,1.0114,97.38,16,91,10.74,overcast clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
4,4,Tessalit,ML,20.1986,1.0114,97.38,16,91,10.74,overcast clouds
5,5,Vercelli,IT,45.3231,8.4153,78.62,67,47,5.53,scattered clouds
8,8,Jamestown,US,42.0970,-79.2353,77.18,65,1,1.99,clear sky
11,11,Cabedelo,BR,-6.9811,-34.8339,75.87,84,11,11.77,few clouds
14,14,Mitsamiouli,KM,-11.3847,43.2844,77.00,70,12,9.60,few clouds
17,17,Bonthe,SL,7.5264,-12.5050,76.69,88,87,6.53,light rain
18,18,Georgetown,MY,5.4112,100.3354,79.90,86,20,2.30,haze
19,19,Korla,CN,41.7597,86.1469,75.25,40,82,2.28,broken clouds
21,21,Muisne,EC,0.6000,-80.0333,75.88,86,78,7.40,broken clouds
24,24,Victoria,HK,22.2855,114.1577,81.32,93,96,5.01,light rain


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 4 to 564
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City_ID           203 non-null    int64  
 1   City              203 non-null    object 
 2   Country           203 non-null    object 
 3   Lat               203 non-null    float64
 4   Lng               203 non-null    float64
 5   Max Temp          203 non-null    float64
 6   Humidity          203 non-null    int64  
 7   Cloudiness        203 non-null    int64  
 8   Wind Speed        203 non-null    float64
 9   City Description  203 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 17.4+ KB


In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 4 to 564
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City_ID           203 non-null    int64  
 1   City              203 non-null    object 
 2   Country           203 non-null    object 
 3   Lat               203 non-null    float64
 4   Lng               203 non-null    float64
 5   Max Temp          203 non-null    float64
 6   Humidity          203 non-null    int64  
 7   Cloudiness        203 non-null    int64  
 8   Wind Speed        203 non-null    float64
 9   City Description  203 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 17.4+ KB


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "City Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
4,Tessalit,ML,97.38,overcast clouds,20.1986,1.0114,
5,Vercelli,IT,78.62,scattered clouds,45.3231,8.4153,
8,Jamestown,US,77.18,clear sky,42.0970,-79.2353,
11,Cabedelo,BR,75.87,few clouds,-6.9811,-34.8339,
14,Mitsamiouli,KM,77.00,few clouds,-11.3847,43.2844,
17,Bonthe,SL,76.69,light rain,7.5264,-12.5050,
18,Georgetown,MY,79.90,haze,5.4112,100.3354,
19,Korla,CN,75.25,broken clouds,41.7597,86.1469,
21,Muisne,EC,75.88,broken clouds,0.6000,-80.0333,
24,Victoria,HK,81.32,light rain,22.2855,114.1577,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except(KeyError, IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
                 
#clean_hotel_df.info()
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 5 to 564
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   City              180 non-null    object 
 1   Country           180 non-null    object 
 2   Max Temp          180 non-null    float64
 3   City Description  180 non-null    object 
 4   Lat               180 non-null    float64
 5   Lng               180 non-null    float64
 6   Hotel Name        180 non-null    object 
dtypes: float64(3), object(4)
memory usage: 11.2+ KB


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.head()

,City,Country,Max Temp,City Description,Lat,Lng,Hotel Name
5,Vercelli,IT,78.62,scattered clouds,45.3231,8.4153,Hotel Cinzia Ristorante
8,Jamestown,US,77.18,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
11,Cabedelo,BR,75.87,few clouds,-6.9811,-34.8339,Hotel Pigalle
14,Mitsamiouli,KM,77.00,few clouds,-11.3847,43.2844,Foyer ADM
17,Bonthe,SL,76.69,light rain,7.5264,-12.5050,Bonthe Holiday Village


In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

###### 11a. Add a marker layer for each city to the map. 

##### 11b. Display the figure

In [26]:
# 11a
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b
fig

Figure(layout=FigureLayout(height='420px'))